## Downscaling with the DeepESD model (stochastic)

This notebook showcases a simple application of deep4downscaling for the stochastic (probabilistic) statistical downscaling of precipitation. To do so, we will implement the following actions:

- Define and train the DeepESD architecture [1] using a probabilistic loss function [2].
- Downscale and evaluate results over a test period.
- Downscale outputs from a Global Climate Model (GCM).
- Generate the corresponding downscaled climate change signals.

### Train the model

In [1]:
DATA_PATH = './data/input'
FIGURES_PATH = './figures'
MODELS_PATH = './models'
ASYM_PATH = './data/asym'

When working with climate data, xarray is an essential library, and deep4downscaling heavily relies on it. For the deep learning component, deep4downscaling uses PyTorch, one of the most popular frameworks in the field.

In [2]:
import xarray as xr
import torch
from torch.utils.data import DataLoader, random_split

import sys; sys.path.append('/home/jovyan/deep4downscaling')
import deep4downscaling.viz
import deep4downscaling.trans
import deep4downscaling.deep.loss
import deep4downscaling.deep.utils
import deep4downscaling.deep.models
import deep4downscaling.deep.train
import deep4downscaling.deep.pred
import deep4downscaling.metrics
import deep4downscaling.metrics_ccs

We will begin by loading the predictor. In this case, we select various large-scale variables from ERA5 at different height levels. These variables are already stored in a NetCDF file, the standard data format for deep4downscaling. Unfortunately, due to GitHub's size restrictions, we are unable to upload these files to the repository. However, the following cells provide an overview of the data, making it straightforward to reproduce this notebook with a similar file.

In [3]:
# Load predictors
predictor_filename = f'{DATA_PATH}/ERA5_NorthAtlanticRegion_1-5dg_full.nc'
predictor = xr.open_dataset(predictor_filename)

In [4]:
predictor

<xarray.Dataset> Size: 1GB
Dimensions:  (lon: 42, lat: 31, time: 16071)
Coordinates:
  * lon      (lon) float64 336B -39.0 -37.5 -36.0 -34.5 ... 18.0 19.5 21.0 22.5
  * lat      (lat) float64 248B 23.5 25.0 26.5 28.0 29.5 ... 64.0 65.5 67.0 68.5
  * time     (time) datetime64[ns] 129kB 1979-01-01 1979-01-02 ... 2022-12-31
Data variables: (12/13)
    t500     (time, lat, lon) float32 84MB ...
    t700     (time, lat, lon) float32 84MB ...
    t850     (time, lat, lon) float32 84MB ...
    q500     (time, lat, lon) float32 84MB ...
    q700     (time, lat, lon) float32 84MB ...
    q850     (time, lat, lon) float32 84MB ...
    ...       ...
    v700     (time, lat, lon) float32 84MB ...
    v850     (time, lat, lon) float32 84MB ...
    u500     (time, lat, lon) float32 84MB ...
    u700     (time, lat, lon) float32 84MB ...
    u850     (time, lat, lon) float32 84MB ...
    msl      (time, lat, lon) float32 84MB ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-06-06 07:20:26 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

The deep4downscaling library provides several functions to facilitate an initial visualization of the data. For example, the `deep4downscaling.viz.multiple_map_plot` function allows you to visualize an `xarray.Dataset`. These functions rely on matplotlib and cartopy. By default, the figure is saved as a `.pdf` file in the path specified by the `output_path argument`.

In [5]:
deep4downscaling.viz.multiple_map_plot(data=predictor.mean('time'),
                                       output_path=f'./{FIGURES_PATH}/predictor_climatology.pdf')

The predictand is an `xarray.Dataset` containing a single variable (the target). In this notebook, we will focus on downscaling accumulated precipitation over the region of peninsular Spain and the Balearic Islands.

In [6]:
predictand_filename = f'{DATA_PATH}/pr_AEMET.nc'
predictand = xr.open_dataset(predictand_filename)

In [7]:
predictand

<xarray.Dataset> Size: 10GB
Dimensions:  (lon: 400, lat: 251, time: 25933)
Coordinates:
  * lon      (lon) float64 3kB -13.18 -13.12 -13.07 -13.02 ... 6.675 6.725 6.775
  * lat      (lat) float64 2kB 33.48 33.52 33.57 33.62 ... 45.87 45.92 45.97
  * time     (time) datetime64[ns] 207kB 1951-01-01 1951-01-02 ... 2021-12-31
Data variables:
    pr       (time, lat, lon) float32 10GB ...

Similar to the predictors, deep4downscaling can also be used for an initial visualization of the predictand.

In [8]:
day_to_viz = '10-04-2015'
deep4downscaling.viz.simple_map_plot(data=predictand.sel(time=day_to_viz),
                                     colorbar='hot_r', var_to_plot='pr',
                                     output_path=f'./{FIGURES_PATH}/predictand_day.pdf')

Deep4downscaling also includes several common preprocessing techniques used in statistical downscaling, such as removing NaN values, aligning datasets (e.g., across time), bias adjustment, and standardization, among others.

In [9]:
# Remove days with nans in the predictor
predictor = deep4downscaling.trans.remove_days_with_nans(predictor)

# Align both datasets in time
predictor, predictand = deep4downscaling.trans.align_datasets(predictor, predictand, 'time')

There are no observations containing null values


To adhere to the standard training/validation scheme in the machine learning field, we divide the predictors and predictand into training and test sets.

In [10]:
years_train = ('1980', '2010')
years_test = ('2011', '2020')

x_train = predictor.sel(time=slice(*years_train))
y_train = predictand.sel(time=slice(*years_train))

x_test = predictor.sel(time=slice(*years_test))
y_test = predictand.sel(time=slice(*years_test))

Before feeding the predictors to the deep learning model, we standardize them to have a mean of zero and a standard deviation of one. This is done using the `deep4downscaling.trans.standardize` function.

In [11]:
x_train_stand = deep4downscaling.trans.standardize(data_ref=x_train, data=x_train)

For training and inference, the data will be transformed into the torch.Tensor type. To facilitate the transition from NetCDF to torch.Tensor, especially when computing projections (predictions), we define a mask around the predictand to use throughout the entire workflow.

In [12]:
y_mask = deep4downscaling.trans.compute_valid_mask(y_train) 

All deep learning models implemented in deep4downscaling flatten their output into a vector, standardizing its dimensions to the shape `(time, grid point)`.

In [13]:
y_train_stack = y_train.stack(gridpoint=('lat', 'lon'))
y_mask_stack = y_mask.stack(gridpoint=('lat', 'lon'))

The DeepESD architecture consists of a set of convolutional layers followed by a final dense layer. In our case, since the predictand contains NaN values for sea grid points, we filter out these grid points to save computation. This reduces the number of neurons in the final fully connected layer. By applying this operation using the mask, the conversion between the model's output and the corresponding NetCDF becomes straightforward.

In [14]:
y_mask_stack_filt = y_mask_stack.where(y_mask_stack==1, drop=True)
y_train_stack_filt = y_train_stack.where(y_train_stack['gridpoint'] == y_mask_stack_filt['gridpoint'],
                                             drop=True)

The following cell applies the recommended transformation for training a model using the negative log-likelihood of a Bernoulli-Gamma distribution. The key idea is to set a threshold that determines which days are used to train the Gamma distribution. This allows the deep learning model to learn the Gamma component only from days that meet this condition (typically by excluding dry days).

In [18]:
threshold_pr = 0.1
y_train_stack_filt = deep4downscaling.deep.utils.precipitation_NLL_trans(data=y_train_stack_filt,
                                                                         threshold=threshold_pr)

The deep4downscaling library includes various loss functions for training deep learning models. In this notebook, following [1, 2], we focus on minimizing the negative log-likelihood of a Bernoulli-Gamma distribution. Instead of directly modeling precipitation, this approach models the parameters of two separate distributions: a Bernoulli distribution for precipitation occurrence, and a Gamma distribution for its amount.

To compute the downscaled projections, we sample from the learned distributions. Training the model in this way allows it to be probabilistic, as it produces a full probability distribution that models the conditional behavior of precipitation given the state of the predictors.

In [19]:
loss_function = deep4downscaling.deep.loss.NLLBerGammaLoss(ignore_nans=False)

NetCDF is not well-suited for use with PyTorch (or for converting to the `torch.Tensor` type). In contrast, NumPy is.

In [20]:
x_train_stand_arr = deep4downscaling.trans.xarray_to_numpy(x_train_stand)
y_train_arr = deep4downscaling.trans.xarray_to_numpy(y_train_stack_filt)

With our data now in the numpy format, we can create the `torch.Dataset` and `torch.DataLoader` to feed batches of data to the deep learning model during training.

In [21]:
# Create Dataset
train_dataset = deep4downscaling.deep.utils.StandardDataset(x=x_train_stand_arr,
                                                            y=y_train_arr)

# Split into training and validation sets
train_dataset, valid_dataset = random_split(train_dataset,
                                            [0.9, 0.1])

# Create DataLoaders
batch_size = 64

train_dataloader = DataLoader(train_dataset, batch_size=batch_size,
                              shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size,
                              shuffle=True)

Deep4downscaling includes several predefined deep learning architectures (e.g., DeepESD and U-Net), but custom architectures can be easily defined using the standard PyTorch framework. However, because deep4downscaling relies on a final flattening operation (as mentioned earlier), we recommend reviewing the implementations in `deep4downscaling.deep.models` and using them as a foundation.

While deep4downscaling lacks a formal documentation page, all its functions and arguments are properly documented within the code.

In [22]:
?deep4downscaling.deep.models.DeepESDpr

Init signature:
deep4downscaling.deep.models.DeepESDpr(
    x_shape: tuple,
    y_shape: tuple,
    filters_last_conv: int,
    stochastic: bool,
    last_relu: bool = False,
)
Docstring:     
DeepESD model as proposed in Baño-Medina et al. 2024 for precipitation
downscaling. This implementation allows for a deterministic (MSE-based)
and stochastic (NLL-based) definition.

Baño-Medina, J., Manzanas, R., Cimadevilla, E., Fernández, J., González-Abad,
J., Cofiño, A. S., and Gutiérrez, J. M.: Downscaling multi-model climate projection
ensembles with deep learning (DeepESD): contribution to CORDEX EUR-44, Geosci. Model
Dev., 15, 6747–6758, https://doi.org/10.5194/gmd-15-6747-2022, 2022.

Parameters
----------
x_shape : tuple
    Shape of the data used as predictor. This must have dimension 4
    (time, channels/variables, lon, lat).

y_shape : tuple
    Shape of the data used as predictand. This must have dimension 2
    (time, gridpoint)

filters_last_conv : int
    Number of filters/kern

In this notebook, we will train the DeepESD architecture with a single final convolutional layer. Since we are minimizing the negative log-likelihood of a Bernoulli-Gamma distribution, we need to set the stochastic parameter to True. This is because the model must learn multiple final fully-connected layers, one for each parameter of the distribution being modeled.

In [23]:
model_name = 'deepesd_pr'
model = deep4downscaling.deep.models.DeepESDpr(x_shape=x_train_stand_arr.shape,
                                               y_shape=y_train_arr.shape,
                                               filters_last_conv=1,
                                               stochastic=True)

We set the typical training hyperparameters, as is commonly done in PyTorch.

In [24]:
num_epochs = 10000
patience_early_stopping = 20

learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(),
                             lr=learning_rate)

Deep learning models can run on either CPU or GPU devices. We provide the corresponding `.yml` environment files (`deep4downscaling/requirement`) to set up a basic Conda environment for running deep4downscaling.

In [25]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

Deep4downscaling provides the `deep4downscaling.deep.train.standard_training_loop`, which implements a basic training routine. Models are saved based on their performance on a validation set through an early stopping process, with the final saved model being the one that achieves the best score on this set. To disable early stopping, you can pass `None` to the `patience_early_stopping` argument. We recommend users consult the `?deep4downscaling.deep.train.standard_training_loop` for further details about this function.

In [26]:
train_loss, val_loss = deep4downscaling.deep.train.standard_training_loop(
                            model=model, model_name=model_name, model_path=MODELS_PATH,
                            device=device, num_epochs=num_epochs,
                            loss_function=loss_function, optimizer=optimizer,
                            train_data=train_dataloader, valid_data=valid_dataloader,
                            patience_early_stopping=patience_early_stopping)

Epoch 1 (12.87 secs) | Training Loss 1.6042 Valid Loss 1.4261 (Model saved)
Epoch 2 (12.56 secs) | Training Loss 1.3869 Valid Loss 1.311 (Model saved)
Epoch 3 (12.44 secs) | Training Loss 1.309 Valid Loss 1.255 (Model saved)
Epoch 4 (12.45 secs) | Training Loss 1.2633 Valid Loss 1.2249 (Model saved)
Epoch 5 (12.47 secs) | Training Loss 1.2369 Valid Loss 1.2085 (Model saved)
Epoch 6 (12.46 secs) | Training Loss 1.2201 Valid Loss 1.1937 (Model saved)
Epoch 7 (12.44 secs) | Training Loss 1.2059 Valid Loss 1.1787 (Model saved)
Epoch 8 (12.46 secs) | Training Loss 1.1947 Valid Loss 1.1725 (Model saved)
Epoch 9 (12.47 secs) | Training Loss 1.1827 Valid Loss 1.1683 (Model saved)
Epoch 10 (12.56 secs) | Training Loss 1.1772 Valid Loss 1.1644 (Model saved)
Epoch 11 (13.29 secs) | Training Loss 1.168 Valid Loss 1.1577 (Model saved)
Epoch 12 (13.75 secs) | Training Loss 1.1633 Valid Loss 1.1504 (Model saved)
Epoch 13 (12.42 secs) | Training Loss 1.1577 Valid Loss 1.1436 (Model saved)
Epoch 14 (12

### Downscale the test set

Once a model has been trained and saved as a `.pt` file, it is easy to compute predictions on a new set of predictors. In this example, we will compute predictions on the test set, which was subset a few cells above. It is important to standardize the test data using the mean and standard deviation computed from the training set.

In [28]:
# Load the model weights into the DeepESD architecture
model.load_state_dict(torch.load(f'{MODELS_PATH}/{model_name}.pt'))

# Standardize
x_test_stand = deep4downscaling.trans.standardize(data_ref=x_train, data=x_test)

Since the model doesn't predict precipitation directly, but rather the parameters of the distribution that generates it, we need to sample from this distribution to produce a conditional projection. To do this, we use the `deep4downscaling.deep.pred.compute_preds_ber_gamma function` (notice that something equivalent exists for temperature downscaling).

In [29]:
# Compute predictions
pred_test = deep4downscaling.deep.pred.compute_preds_ber_gamma(x_data=x_test_stand, model=model,
                                                               threshold=threshold_pr, ensemble_size=1, # It is possible to sample multiple times from the conditional distribution
                                                               device=device, var_target='pr',
                                                               mask=y_mask, batch_size=16)

In [33]:
# Visualize the predictions
deep4downscaling.viz.simple_map_plot(data=pred_test.mean('time'),
                                     colorbar='hot_r', var_to_plot='pr',
                                     output_path=f'./{FIGURES_PATH}/prediction_test_mean_sto.pdf')

The `deep4downscaling.metrics` module, included within deep4downscaling, implements various metrics commonly used to assess deep learning models in the context of statistical downscaling. These include biases of different indices, spatial and probabilistic metrics, and multivariate indices, among others. In this example, we demonstrate its use by computing the relative bias of the Rx1day index between the target (test set) and the predictions for the winter months.

In [34]:
bias_rel_rx1day = deep4downscaling.metrics.bias_rel_rx1day(target=y_test, pred=pred_test,
                                                           var_target='pr', season='winter') 

In [35]:
print(bias_rel_rx1day)

<xarray.Dataset> Size: 407kB
Dimensions:  (lat: 251, lon: 400)
Coordinates:
  * lat      (lat) float64 2kB 33.48 33.52 33.57 33.62 ... 45.87 45.92 45.97
  * lon      (lon) float64 3kB -13.18 -13.12 -13.07 -13.02 ... 6.675 6.725 6.775
Data variables:
    pr       (lat, lon) float32 402kB nan nan nan nan nan ... nan nan nan nan


### Downscale a Global Climate Model

Similarly, it is possible to use large-scale variables from a GCM as predictors for the trained model. To do this, we load a NetCDF file formatted in the same way as the one used for the predictor variables. In this example, we use predictors from the EC-Earth3-Veg model (r1i1p1f1).

As we are interested in computing the climate change signal, we will downscale the GCM data for both a historical and a future period. The future period, in this case, corresponds to the SSP370 scenario.

In [36]:
# Load GCM data from the historical period
gcm_hist = xr.open_dataset(f'{DATA_PATH}/EC-Earth3-Veg_r1i1p1f1_ssp370_hist.nc')

In [37]:
gcm_hist

<xarray.Dataset> Size: 2GB
Dimensions:  (lon: 42, lat: 31, time: 23741)
Coordinates:
  * lon      (lon) float64 336B -39.0 -37.5 -36.0 -34.5 ... 18.0 19.5 21.0 22.5
  * lat      (lat) float64 248B 23.5 25.0 26.5 28.0 29.5 ... 64.0 65.5 67.0 68.5
  * time     (time) datetime64[ns] 190kB 1950-01-01 1950-01-02 ... 2014-12-31
Data variables: (12/13)
    t500     (time, lat, lon) float32 124MB ...
    t700     (time, lat, lon) float32 124MB ...
    t850     (time, lat, lon) float32 124MB ...
    q500     (time, lat, lon) float32 124MB ...
    q700     (time, lat, lon) float32 124MB ...
    q850     (time, lat, lon) float32 124MB ...
    ...       ...
    v700     (time, lat, lon) float32 124MB ...
    v850     (time, lat, lon) float32 124MB ...
    u500     (time, lat, lon) float32 124MB ...
    u700     (time, lat, lon) float32 124MB ...
    u850     (time, lat, lon) float32 124MB ...
    msl      (time, lat, lon) float32 124MB ...
Attributes: (12/46)
    CDI:                                Climate Data Interface version 1.9.10...
    source:                             EC-Earth3-Veg (2019): \naerosol: none...
    institution:                        AEMET, Spain; BSC, Spain; CNR-ISAC, I...
    Conventions:                        CF-1.7 CMIP-6.2
    activity_id:                        CMIP
    branch_method:                      standard
    ...                                 ...
    variable_id:                        ta
    variant_label:                      r1i1p1f1
    license:                            CMIP6 model data produced by EC-Earth...
    cmor_version:                       3.4.0
    tracking_id:                        hdl:21.14100/ae287024-3ebe-467d-8b38-...
    latest_applied_cmor_fixer_version:  v3.0

In [38]:
# Load GCM data from the future period
gcm_fut = xr.open_dataset(f'{DATA_PATH}/EC-Earth3-Veg_r1i1p1f1_ssp370_fut.nc')

In [39]:
gcm_fut

<xarray.Dataset> Size: 2GB
Dimensions:  (lon: 42, lat: 31, time: 31411)
Coordinates:
  * lon      (lon) float64 336B -39.0 -37.5 -36.0 -34.5 ... 18.0 19.5 21.0 22.5
  * lat      (lat) float64 248B 23.5 25.0 26.5 28.0 29.5 ... 64.0 65.5 67.0 68.5
  * time     (time) datetime64[ns] 251kB 2015-01-01 2015-01-02 ... 2100-12-31
Data variables: (12/13)
    t500     (time, lat, lon) float32 164MB ...
    t700     (time, lat, lon) float32 164MB ...
    t850     (time, lat, lon) float32 164MB ...
    q500     (time, lat, lon) float32 164MB ...
    q700     (time, lat, lon) float32 164MB ...
    q850     (time, lat, lon) float32 164MB ...
    ...       ...
    v700     (time, lat, lon) float32 164MB ...
    v850     (time, lat, lon) float32 164MB ...
    u500     (time, lat, lon) float32 164MB ...
    u700     (time, lat, lon) float32 164MB ...
    u850     (time, lat, lon) float32 164MB ...
    msl      (time, lat, lon) float32 164MB ...
Attributes: (12/45)
    CDI:                    Climate Data Interface version 1.9.10 (https://mp...
    source:                 EC-Earth3-Veg (2019): \naerosol: none\natmos: IFS...
    institution:            AEMET, Spain; BSC, Spain; CNR-ISAC, Italy; DMI, D...
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP AerChemMIP
    branch_method:          standard
    ...                     ...
    title:                  EC-Earth3-Veg output prepared for CMIP6
    variable_id:            ta
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by EC-Earth-Consortium ...
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/a821b907-94f2-478f-a162-1a8ff7030fba

Before being passed to the deep learning model, GCM predictors are first bias-corrected to increase their similarity to the ERA5 predictors used during training, thus avoiding a distributional shift. To achieve this, deep4downscaling implements the scaling delta correction method suggested in [3, 4] through the `deep4downscaling.trans.scaling_delta_correction` function. After applying this correction, and as with the test data, we standardize the predictors using the mean and standard deviation computed from the training set.

In [40]:
# Bias adjust the GCM predictors
gcm_hist_corrected = deep4downscaling.trans.scaling_delta_correction(data=gcm_hist,
                                                                     gcm_hist=gcm_hist, obs_hist=x_train)
gcm_fut_corrected = deep4downscaling.trans.scaling_delta_correction(data=gcm_fut,
                                                                    gcm_hist=gcm_hist, obs_hist=x_train)

# Standardize
gcm_hist_corrected_stand = deep4downscaling.trans.standardize(data_ref=x_train, data=gcm_hist_corrected)
gcm_fut_corrected_stand = deep4downscaling.trans.standardize(data_ref=x_train, data=gcm_fut_corrected)

We then compute the predictions for the historical and future periods as usual.

In [41]:
proj_historical = deep4downscaling.deep.pred.compute_preds_ber_gamma(
                    x_data=gcm_hist_corrected_stand, model=model,
                    threshold=threshold_pr, ensemble_size=1,
                    device=device, var_target='pr',
                    mask=y_mask, batch_size=16)

proj_future = deep4downscaling.deep.pred.compute_preds_ber_gamma(
                    x_data=gcm_fut_corrected_stand, model=model,
                    threshold=threshold_pr, ensemble_size=1,
                    device=device, var_target='pr',
                    mask=y_mask, batch_size=16)

### Compute climate change signals

After computing the downscaled projections from the GCM, we can calculate the corresponding climate change signal using the `deep4downscaling.metrics_ccs` module.

In [42]:
reduction_function = deep4downscaling.metrics_ccs.mean
ccs_mean = deep4downscaling.metrics_ccs.compute_ccs(hist_data=proj_historical, fut_data=proj_future,
                                                    reduction_function=reduction_function,
                                                    relative=True)

In [43]:
# Visualize the climate change signal
deep4downscaling.viz.simple_map_plot(data=ccs_mean,
                                     colorbar='BrBG', var_to_plot='pr',
                                     vlimits=(-40, 40), num_levels=16,
                                     output_path=f'./{FIGURES_PATH}/ccs_mean_sto.pdf')

### References

[1] Baño-Medina, J., Manzanas, R., Cimadevilla, E., Fernández, J., González-Abad, J., Cofiño, A. S., & Gutiérrez, J. M. (2022). Downscaling multi-model climate projection ensembles with deep learning (DeepESD): Contribution to CORDEX EUR-44. Geoscientific Model Development Discussions, 2022, 1-14.

[2] Baño-Medina, J., Manzanas, R., & Gutiérrez, J. M. (2020). Configuration and intercomparison of deep learning neural models for statistical downscaling. Geoscientific Model Development, 13(4), 2109-2124.

[3] González-Abad, J., & Gutiérrez, J. M. (2024). Are Deep Learning Methods Suitable for Downscaling Global Climate Projections? Review and Intercomparison of Existing Models. arXiv preprint arXiv:2411.05850.

[4] Baño-Medina, J., Manzanas, R., & Gutiérrez, J. M. (2021). On the suitability of deep convolutional neural networks for continental-wide downscaling of climate change projections. Climate Dynamics, 57(11), 2941-2951.